In [288]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from math import exp, sqrt

In [289]:
df = pd.read_csv("chips.csv")
df['class'].replace({'P': 1, 'N': -1}, inplace=True)

In [290]:
def linear(x, y):
    return x @ y


kernels = [linear]

for p in range(2, 5 + 1):
    name = f'polynomial_{p}'
    exec(f'def {name}(x, y): return (1 + x @ y)**{p}')
    exec(f'kernels.append({name})')

for b in range(1, 5 + 1):
    name = f'guassian_{b}'
    exec(f'def {name}(x, y): return exp({b} * np.linalg.norm(x - y)**2)')
    exec(f'kernels.append({name})')

In [291]:
a = np.array([1, 1])
b = np.array([3, 2])

In [292]:
from sklearn.model_selection import train_test_split

X = df[['x', 'y']]
y = df[['class']]

a = X.to_numpy()[0]
b = X.to_numpy()[1]

linear(a,b)
X[:3].to_numpy()[0]

array([0.051267, 0.69956 ])

In [293]:
from subprocess import run, PIPE
from sys import stdout
from io import StringIO

class SVM:
    def __init__(self, kernel, C):
        self.kernel = kernel
        self.C = C
        self.X_train = None
        self.y_train = None
        self.lambdas = None
        self.b = None
        self.distances = None
        self.io = StringIO()


    def calc_matrix(self, X):
        n = X.shape[0]
        self.distances = np.fromfunction(np.vectorize(lambda i, j: self.kernel(X[i], X[j])), (n, n), dtype=int)


    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        self.calc_matrix(X)

        np.savetxt(self.io, np.append(self.distances, y, axis=1), fmt='%.8f')
        args = str(self.distances.shape[0]) + '\n' + self.io.getvalue() + str(self.C)
        # print(args)
        *self.lambdas, self.b  = map(float, run(['smo.exe'], stdout=PIPE, input=args, encoding='ascii').stdout.split('\n'))
        print(self.lambdas, self.b)
        return self
        

    def predict(self, X):
        res = self.b
        for l, yi, x in zip(self.lambdas, y_train, self.X_train):
            res += l * yi * self.kernel(X, x)
        return res

svm = SVM(polynomial_4, 100)
X_train = X.to_numpy()
y_train = y.to_numpy()
svm.fit(X_train, y_train)
svm.predict(X_train[0])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 100.0, 86.1555, 100.0, 99.9528, 22.0492, 0.0, 100.0, 100.0, 8.11602, 0.0, 100.0, 0.0, 56.9632, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0.0, 100.0, 0.0, 73.624, 100.0, 43.6272, 0.0, 0.0, 0.0, 0.0, 42.604, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.9913, 0.0, 100.0, 0.0, 100.0, 100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 100.0, 100.0, 0.0, 0.0, 100.0, 4.96223, 0.0, 77.4661, 0.0, 0.0, 24.5195, 0.0, 0.0, 0.0, 7.57128, 18.2297, 100.0, 100.0, 100.0, 0.0, 0.0, 100.0, 100.0, 0.0, 100.0, 0.0, 0.0, 25.2982, 0.0, 100.0, 65.5997, 0.0, 100.0, 100.0, 67.3207, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 100.0, 0.0, 100.0, 100.0, 46.5236, 0.0, 0.0, 0.0, 0.0, 2.59218, 100.0, 0.0, 0.0, 100.0] 3.04335


array([1.50926723])

In [294]:
from subprocess import Popen, PIPE, run

s='6 5 4 6 9 11 10 -1 4 5 6 9 10 11 -1 6 6 8 12 14 14 -1 9 9 12 18 21 21 1 11 10 14 21 25 24 1 10 11 14 21 24 25 1 1'
run(['smo.exe'], stdout=PIPE, input=s, encoding='ascii').stdout.split('\n')

['0', '0', '1', '1', '0', '0', '-5']